## Import ##

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
import cv2 as cv
import json
import pandas as pd


In [2]:
def flatten(l):
    return [item for sublist in l for item in sublist]


def get_txt_data(path: Path) -> np.ndarray:
    with path.open("r", encoding="utf-8") as f:
        result: list = json.loads(f.readline())
        return np.array(flatten(result))


p = pathlib.Path('../../openCV_python/data/out/resized_frames')
txt_files = list(p.glob('*.txt'))
img_files = [item.parent.resolve() / item.stem for item in txt_files]
txt_files[0].stem
image = txt_files[0].parent.resolve() / txt_files[0].stem

img_count = len(txt_files)
img_shape = cv.imread(str(image)).shape

txt_shape = get_txt_data(txt_files[0]).shape

input_shape = (img_count, *img_shape)
output_shape = (img_count, *txt_shape)
input_data = np.zeros(input_shape)
output_data = np.zeros(output_shape)
data = []

for i in range(len(img_files)):
    input_data[i] = cv.imread(str(img_files[i]))
    output_data[i] = get_txt_data(txt_files[i])

print('input shape')
print(input_shape)
print()
print('output shape')
print(output_shape)


def load_data():
    train_images = input_data[0:154]
    train_labels = output_data[0:154]

    test_images = input_data[154:254]
    test_labels = output_data[154:254]

    return (train_images, train_labels), (test_images, test_labels)


input shape
(254, 793, 560, 3)

output shape
(254, 8)


In [3]:
(train_images, train_labels), (test_images, test_labels) = load_data()
# (train_images, train_labels), (test_images,
#                                test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
# train_images, test_images = train_images / 255.0, test_images / 255.0


In [4]:
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']

# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i])
#     # The CIFAR labels happen to be arrays, 
#     # which is why you need the extra index
#     plt.xlabel(class_names[train_labels[i][0]])
# plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(793, 560, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(8))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss="mae",)

history = model.fit(
    train_images, train_labels, 
    epochs=50, 
    batch_size=500,
    validation_data=(test_images, test_labels)
    )

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
# print(test_acc)